# MachineLearningModel.ipynb
In this notebook we will create a machine learning model using for Texture Recognition using Haralick Texture Features  


Authors: Alexander Goudemond (219030365) and Sumeith Ishwanthlal (219006284)

In [5]:
import cv2
import numpy as np
from os import getcwd, walk
import glob
from mahotas import features
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt


## Data Preprocessing

In [6]:
def extract_features(image):
        textures = features.haralick(image)
        hf_mean = textures.mean(axis=0)
        return hf_mean

In [ ]:
train_features = []
train_labels = []
currentDir = getcwd()
photoPath = currentDir + "\\Notes_DataSet"
path = walk(photoPath)
numProcessedImages = 0
print("Started to extract haralick features from images")
for root, directories, files in path:
    for file in files:
        image = cv2.imread("Notes_DataSet" + "\\" + file, cv2.IMREAD_GRAYSCALE)
        haralick_features = extract_features(image)
        train_features.append(features)
        if "010" in file.split("_"):
            train_labels.append("R10") 
            
        if "020" in file.split("_"):
            train_labels.append("R20")

        if "050" in file.split("_"):
            train_labels.append("R30")
        
        if "100" in file.split("_"):
            train_labels.append("R40")
        
        if "200" in file.split("_"):
            train_labels.append("R50")
        numProcessedImages += 1

        if numProcessedImages % 5 == 0:
            print(f"Processed {numProcessedImages} images...")
        

In [ ]:
currentDir = getcwd()
    photoPath = currentDir + "\\Notes_DataSet"
    path = walk(photoPath)

    # 4 X 2D Array: Image Sizes, Min/Max Values, Num Pics
    dataSetInfo = getDataSetInfo(path)

    spacers = "-" * 60

    print(spacers, "List of Image Sizes:", spacers, sep="\n")
    for item in dataSetInfo[0]:
        print(item[1])

    print("", spacers, "Global minimum and maximum dimensions:", spacers, sep="\n")
    for item in dataSetInfo[1]:
        print(item[0], item[1])

    print("", spacers, "Average X and Y Values:", spacers, sep="\n")
    for item in dataSetInfo[2]:
        print(item[0], item[1])

    print("", spacers, "Total Number of Pictures in DataSet:", spacers, sep="\n")
    print(dataSetInfo[3][0][0], dataSetInfo[3][0][1])